In [1]:
import tensorflow as tf
from tf_agents.agents.ppo import ppo_agent
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.environments import parallel_py_environment
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.networks import actor_distribution_rnn_network
from tf_agents.networks import value_network
from tf_agents.networks import value_rnn_network
from tf_agents.policies import policy_saver
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.utils import common

import gym
import gym_solventx 
import copy

In [2]:
tf_env = tf_py_environment.TFPyEnvironment(suite_gym.load('gym_solventx-v0'))

In [3]:
saved_policy = tf.compat.v2.saved_model.load('run6\policy_000007500')#'run1\policy_000011500','run3\policy_000015500','run2\policy_000016000'


In [4]:
n_episodes = 50
returns = []
recoverys = []
puritys = []
gross_action_stats ={action_id:0 for action_id in range(23) }
use_rnn = False
for i in range(n_episodes):
    time_step = tf_env.reset()
    policy_state = saved_policy.get_initial_state(tf_env.batch_size)
    rewards= []
    action_sequence = []
    final_action_stats ={action_id:0 for action_id in range(23) }
    initial_state = copy.copy(tf_env.pyenv.envs[0]._env.obj.variables)
    #initial_recovery = copy.copy(tf_env.pyenv.envs[0]._env.obj.strip_recov[0])
    initial_recovery = copy.copy(tf_env.pyenv.envs[0]._env.obj.recovery['Strip-0'][tf_env.pyenv.envs[0]._env.obj.ree.index('Nd')])
    #initial_purity = copy.copy(tf_env.pyenv.envs[0]._env.obj.strip_pur[0])
    initial_purity =  copy.copy(tf_env.pyenv.envs[0]._env.obj.purity['Strip-0'][tf_env.pyenv.envs[0]._env.obj.ree.index('Nd')])
    
    while not time_step.is_last():
        if use_rnn:
            action_step, policy_state, _ = saved_policy.action(time_step, policy_state) 
            action = action_step.numpy()[0]
            time_step = tf_env.step(action_step)
        else:
            action_step = saved_policy.action(time_step)
            action = action_step.action.numpy()[0]
            time_step = tf_env.step(action_step.action)            
        
        gross_action_stats.update({action:gross_action_stats[action]+1})
        final_action_stats.update({action:final_action_stats[action]+1})

        print('Step:',tf_env.pyenv.envs[0]._env.steps)
        print('Action:',action)
        action_sequence.append(action)
        tf_env.pyenv.envs[0]._env.decipher_action(action)
        reward = time_step.reward.numpy()[0]    
        print('Reward:',reward)
        tf_env.pyenv.envs[0]._env.show_design_performance()

        rewards.append(reward)
    final_state = tf_env.pyenv.envs[0]._env.obj.variables
    final_recovery = tf_env.pyenv.envs[0]._env.obj.recovery['Strip-0'][tf_env.pyenv.envs[0]._env.obj.ree.index('Nd')]
    final_purity = tf_env.pyenv.envs[0]._env.obj.purity['Strip-0'][tf_env.pyenv.envs[0]._env.obj.ree.index('Nd')]
    recoverys.append(final_recovery)
    puritys.append(final_purity)
    returns.append(sum(rewards))
    print(f'Return at episode {i}:{returns[-1]}')
    
    if final_purity > 0.75 and final_recovery > 0.4:
        print(f'Design objective achieved at episode {i} - terminanting evaluation!')
        
        break

print(f'Initial design:')
tf_env.pyenv.envs[0]._env.show_process_design(initial_state)
print(f'Purity:{initial_purity:.4f}')
print(f'Recovery:{initial_recovery:.4f}')
print(f'Final design:')
tf_env.pyenv.envs[0]._env.show_process_design(final_state)
tf_env.pyenv.envs[0]._env.show_design_performance()

print(f'Average purity:{sum(puritys)/(i+1):.4f}')
print(f'Average recovery:{sum(recoverys)/(i+1):.4f}')
print(f'Average return:{sum(returns)/(i+1):.4f}')

Step: 1
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.265916
Purity:0.7340835360403368
Recovery:0.37152918923321215
Step: 2
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.265916
Purity:0.7340833277228447
Recovery:0.3715493757089133
Step: 3
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.265916
Purity:0.7340831654071456
Recovery:0.3715651093999227
Step: 4
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.265916
Purity:0.73408303883399
Recovery:0.37157737275411723
Step: 5
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.265917
Purity:0.734082940278826
Recovery:0.3715869314598624
Step: 6
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.265917
Purity:0.7340828634382491
Recovery:0.3715943820781003
Step: 7
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.265917
Purity:0.7340828035442419
Recovery:0.37160018960197133
Step: 8
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.265917
Purity:0.734082756863333
Recovery:0.37160471642615145


Step: 1
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.255435
Purity:0.7445642102071715
Recovery:0.8222289503501349
Step: 2
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.255435
Purity:0.7445641769114724
Recovery:0.822229251510178
Step: 3
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.255435
Purity:0.7445641766842
Recovery:0.8222292534849237
Step: 4
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.255435
Purity:0.7445641766819775
Recovery:0.8222292535072312
Step: 5
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.255123
Purity:0.7448767032564788
Recovery:0.8354982735221044
Step: 6
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.254879
Purity:0.7451204169821977
Recovery:0.8455642777189194
Step: 7
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.254689
Purity:0.7453103607815378
Recovery:0.853235549365193
Step: 8
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.254541
Purity:0.7454583545267413
Recovery:0.8591054048785818
Step: 9
Action: 3
Index:1

Step: 99
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.253939
Purity:0.7460602132021247
Recovery:0.8841210013154367
Step: 100
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.253939
Purity:0.7460602132021247
Recovery:0.8841210013154367
Return at episode 1:-25.41137307882309
Step: 1
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.247018
Purity:0.7529811903012276
Recovery:0.2772218034038682
Step: 2
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.251173
Purity:0.7488268421551076
Recovery:0.3681832530747087
Step: 3
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.260877
Purity:0.73912243830245
Recovery:0.4419015407938127
Step: 4
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.260877
Purity:0.7391224535391142
Recovery:0.4419015577649242
Step: 5
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.260877
Purity:0.739122453537801
Recovery:0.44190155777965595
Step: 6
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.260877
Purity:0.7391224535380645
Recovery:0.4419015

Step: 71
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.260855
Purity:0.7391443696162815
Recovery:0.44345961944642026
Step: 72
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.260855
Purity:0.7391443696162815
Recovery:0.44345961944642026
Step: 73
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.260855
Purity:0.7391443696162815
Recovery:0.44345961944642026
Step: 74
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.260855
Purity:0.7391443696162815
Recovery:0.44345961944642026
Step: 75
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.260855
Purity:0.7391443696162815
Recovery:0.44345961944642026
Step: 76
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.260855
Purity:0.7391443696162815
Recovery:0.44345961944642026
Step: 77
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.260855
Purity:0.7391443696162815
Recovery:0.44345961944642026
Step: 78
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.260855
Purity:0.7391443696162815
Recovery:0.44

Step: 36
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.284198
Purity:0.7158017862637054
Recovery:0.22201346845237038
Step: 37
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.284196
Purity:0.715803567744317
Recovery:0.22208693041667205
Step: 38
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.284195
Purity:0.7158049556696968
Recovery:0.22214417377175164
Step: 39
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.284193
Purity:0.7158060437979386
Recovery:0.22218878978835377
Step: 40
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.284193
Purity:0.7158068932681391
Recovery:0.22222356229696794
Step: 41
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.284192
Purity:0.7158075558751034
Recovery:0.2222506638844049
Step: 42
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.284191
Purity:0.7158080770438535
Recovery:0.22227178561101574
Step: 43
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.284191
Purity:0.7158084805854023
Recovery:0.2222

...

Step: 86
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.282407
Purity:0.7175920420180864
Recovery:0.2662467792761795
Step: 87
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.282407
Purity:0.7175920420180864
Recovery:0.2662467792761795
Step: 88
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.282407
Purity:0.7175920420180864
Recovery:0.2662467792761795
Step: 89
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.282407
Purity:0.7175920420180864
Recovery:0.2662467792761795
Step: 90
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.282407
Purity:0.7175920420180864
Recovery:0.2662467792761795
Step: 91
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.282407
Purity:0.7175920420180864
Recovery:0.2662467792761795
Step: 92
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.282407
Purity:0.7175920420180864
Recovery:0.2662467792761795
Step: 93
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.282407
Purity:0.7175920420180864
Recovery:0.266246779

Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.257803
Purity:0.7421968128209037
Recovery:0.621570392782283
Step: 70
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.257803
Purity:0.7421968128209037
Recovery:0.621570392782283
Step: 71
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.257803
Purity:0.7421968128209037
Recovery:0.621570392782283
Step: 72
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.257803
Purity:0.7421968128209037
Recovery:0.621570392782283
Step: 73
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.257803
Purity:0.7421968128209037
Recovery:0.621570392782283
Step: 74
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.257803
Purity:0.7421968128209037
Recovery:0.621570392782283
Step: 75
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.257803
Purity:0.7421968128209037
Recovery:0.621570392782283
Step: 76
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -0.257803
Purity:0.7421968128209037
Recovery:0.621570392782283
Step: 77
Action: 6
Index:3,Variable:H+ Strip-inc


Step: 37
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.259719
Purity:0.7402803780636258
Recovery:0.5985790973859078
Step: 38
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.259556
Purity:0.740443552060118
Recovery:0.6036545022055042
Step: 39
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.259378
Purity:0.7406219396419803
Recovery:0.6073325079185993
Step: 40
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.259165
Purity:0.7408340010778892
Recovery:0.6098383473578427
Step: 41
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.258934
Purity:0.741065031425551
Recovery:0.6114695258149737
Step: 42
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.258716
Purity:0.741283549545742
Recovery:0.6125461387682937
Step: 43
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.258528
Purity:0.741471928390299
Recovery:0.613288025039089
Step: 44
Action: 3
Index:1,Variable:H+ Extraction-dec
Reward: -0.258373
Purity:0.7416268401931636
Recovery:0.61381917906457

Step: 2
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.244261
Purity:0.7557383114089725
Recovery:1.2603257760492875e-10
Step: 3
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.244261
Purity:0.7557383114000545
Recovery:2.713481193656487e-10
Step: 4
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.244261
Purity:0.7557383113929549
Recovery:5.857927374944595e-10
Step: 5
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.244261
Purity:0.7557383113774252
Recovery:1.2685053094564017e-09
Step: 6
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.244261
Purity:0.7557383114682542
Recovery:2.7564466446664867e-09
Step: 7
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.244261
Purity:0.7557383115416285
Recovery:6.0133402731356486e-09
Step: 8
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.244261
Purity:0.755738311702787
Recovery:1.3176957018831197e-08
Step: 9
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.244261
Purity:0.755738312059595
Recovery:2.9020088515335827e-08
Step: 10
Act

Step: 1
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.250053
Purity:0.7499465679532984
Recovery:2.1268458848602247e-15
Step: 2
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.250053
Purity:0.7499465679533065
Recovery:4.510840269588093e-15
Step: 3
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.250053
Purity:0.7499465679533518
Recovery:9.573785616604675e-15
Step: 4
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.250053
Purity:0.7499465679532945
Recovery:2.0335042162032918e-14
Step: 5
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.250053
Purity:0.7499465679533025
Recovery:4.32295766176087e-14
Step: 6
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.250053
Purity:0.7499465679532606
Recovery:9.198970666821394e-14
Step: 7
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.250053
Purity:0.7499465679533199
Recovery:1.959638901418314e-13
Step: 8
Action: 6
Index:3,Variable:H+ Strip-inc
Reward: -1.250053
Purity:0.7499465679534185
Recovery:4.179809218562158e-13
Step: 9
Action:

In [ ]:
final_recovery,final_purity,

In [ ]:
final_state

In [ ]:
gross_action_stats

In [ ]:
-abs(1-2.8334577229110816e-12),min(1, 0.9999999999999973)

In [ ]:
time_step.reward.numpy()[0]

In [ ]:
time_step.is_last(),action_step.action.numpy()[0]

In [ ]:
new_model = tf.keras.models.load_model('solventx_ppo_agent')

In [ ]:
dir(new_model)

In [ ]:
new_model.summary()

In [ ]:
policy =tf.saved_model.load('solventx_ppo_agent')

In [ ]:
time_step = tf_env.reset()
action_step = saved_policy.action(time_step)

In [ ]:
print('Action:',action_step.action.numpy()[0])